## Import libaries

In [1]:
import os
import sys
assert os.path.exists('../src'), f"[ERROR] The path src not detected in the parent directory '{os.getcwd()}'."

if os.getcwd().endswith('/notebooks'):
    os.chdir('..')
    sys.path.append('./src')

print(f'[INFO] Current Directory: "{os.getcwd()}".')

[INFO] Current Directory: "/home/iceking/Desktop/22SS-TUM Lecture Docs/3. Praktikum (IN2106, IN4249)/Repo/topic-modeling-advancements".


In [5]:
%load_ext autoreload
%autoreload 2


from src import preprocessor
from src.utils import load_documents
import nltk
import json
import ast
from time import time
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
combinations = []
with open('combinations.txt') as comb_file:
    for item in comb_file.readlines():
        item = item.replace("'",'"')
        x= ast.literal_eval(item)
        combinations.append(x)

In [ ]:
from src.bulk_runner import OUTPUT_FOLDER
from src.bulk_runner import main_runner
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)
dfs_output_topic_word = []
for configs in combinations:
    try:
        df_output_topic_word = main_runner(configs=configs)
        dfs_output_topic_word.append(df_output_topic_word)
    except Exception:
        print('[WARN] Current execution gave an error!')
        sleep(10)
        continue
pd.concat(dfs_output_topic_word).to_csv(f'./{OUTPUT_FOLDER}/merged.csv')

[INFO] [PREPROCESSOR] Available Preprocessing Functions in the Module:['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_missing_delimiter', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_new_lines', 'remove_extra_spaces', 'remove_html_tags', 'expand_contractions', 'remove_english_stop_words', 'lemmatize', 'lemmatize_verb', 'lemmatize_noun', 'lemmatize_adjective', 'correct_typo']
[INFO] [PREPROCESSOR] Preprocessing starting..
[INFO] [PREPROCESSOR] These string preprocessing methods will be applied to the data in order:
(  'to_lowercase',
   'standardize_accented_chars',
   'remove_url',
   'expand_missing_delimiter',
   'remove_mentions',
   'remove_hashtags',
   'remove_new_lines',
   'remove_html_tags',
   'keep_only_alphabet',
   'remove_extra_spaces')
[INFO] [PREPROCESSOR] Then, these tokenized preprocessing methods will be applied in order:
(  '__tokenize',
   'expand_contractions',
   'remove_english_stop_words',
   'lemmatize_noun',
   'lem

2022-07-24 00:48:24,508 - top2vec - INFO - Pre-processing documents for training
2022-07-24 00:48:24,508 INFO:Pre-processing documents for training


[INFO] [PREPROCESSOR] Preprocessing completed in 15.273 seconds..
[INFO] Running with 4 topics.
[INFO] Top2Vec Parameters:
{
    "embedding_model": "universal-sentence-encoder-multilingual-large",
    "min_count": 10,
    "algorithm": "top2vec",
    "num_topics": 4,
    "doc2vec_speed": "learn",
    "umap_args": {
        "n_neighbors": 15,
        "n_components": [
            3,
            5,
            7
        ],
        "metric": "cosine",
        "random_state": 42
    },
    "hdbscan_args": {
        "min_cluster_size": [
            5,
            10,
            15
        ],
        "metric": "euclidean",
        "cluster_selection_method": "eom"
    },
    "data_name": "crisis_01",
    "run_id": 1658616489
}
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_distiluse-base-multilingual-cased" found, so no need to download.
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_all-MiniLM-L6-v2" found, so no need to download.

2022-07-24 00:48:25,118 - top2vec - INFO - Loading universal-sentence-encoder-multilingual-large model at ./pretrained_models/universal-sentence-encoder-multilingual-large
2022-07-24 00:48:25,118 INFO:Loading universal-sentence-encoder-multilingual-large model at ./pretrained_models/universal-sentence-encoder-multilingual-large
2022-07-24 00:48:26.003477: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-24 00:48:26.003497: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-24 00:48:26.003511: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (iceking-ThinkPad-T490): /proc/driver/nvidia/version does not exist
2022-07-24 00:48:26.003733: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimi